In [1]:
%pip install fastapi uvicorn tensorflow joblib pandas numpy scikit-learn pyngrok nest_asyncio


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for pyngrok from https://files.pythonhosted.org/packages/9e/09/8338c63fe2d92313b7819f38c235746689d88052b112d8717bf45dcb3570/pyngrok-7.2.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import joblib
import numpy as np
import pandas as pd
import pickle
from pyngrok import ngrok
import uvicorn
from tensorflow.keras.models import load_model

# Allow nested event loops
nest_asyncio.apply()

# Define your FastAPI app
app = FastAPI()

# CORS configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],  # Allow requests from this origin
    allow_credentials=True,
    allow_methods=["*"],  # Allow all methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)

# Load models
ann_model = load_model("E:\\Yusra\\SOLAR_MODELS\\ann_model.keras")
rf_model = joblib.load("E:\\Yusra\\SOLAR_MODELS\\rf_model.pkl")
gb_model = joblib.load("E:\\Yusra\\SOLAR_MODELS\\gb_model.pkl")

# Load the scaler
with open("E:\\Yusra\\SOLAR_MODELS\\minmax_scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)

class InputData(BaseModel):
    temperature_2_m_above_gnd: float
    relative_humidity_2_m_above_gnd: float
    mean_sea_level_pressure_MSL: float
    total_precipitation_sfc: float
    snowfall_amount_sfc: float
    total_cloud_cover_sfc: float
    high_cloud_cover_high_cld_lay: int
    medium_cloud_cover_mid_cld_lay: int
    low_cloud_cover_low_cld_lay: int
    shortwave_radiation_backwards_sfc: float
    wind_speed_10_m_above_gnd: float
    wind_direction_10_m_above_gnd: float
    wind_speed_80_m_above_gnd: float
    wind_direction_80_m_above_gnd: float
    wind_gust_10_m_above_gnd: float
    angle_of_incidence: float
    zenith: float
    azimuth: float

# Prediction function
def predict(input_data, model):
    input_df = pd.DataFrame([{
        "temperature_2_m_above_gnd": input_data.temperature_2_m_above_gnd,
        "relative_humidity_2_m_above_gnd": input_data.relative_humidity_2_m_above_gnd,
        "mean_sea_level_pressure_MSL": input_data.mean_sea_level_pressure_MSL,
        "total_precipitation_sfc": input_data.total_precipitation_sfc,
        "snowfall_amount_sfc": input_data.snowfall_amount_sfc,
        "total_cloud_cover_sfc": input_data.total_cloud_cover_sfc,
        "high_cloud_cover_high_cld_lay": input_data.high_cloud_cover_high_cld_lay,
        "medium_cloud_cover_mid_cld_lay": input_data.medium_cloud_cover_mid_cld_lay,
        "low_cloud_cover_low_cld_lay": input_data.low_cloud_cover_low_cld_lay,
        "shortwave_radiation_backwards_sfc": input_data.shortwave_radiation_backwards_sfc,
        "wind_speed_10_m_above_gnd": input_data.wind_speed_10_m_above_gnd,
        "wind_direction_10_m_above_gnd": input_data.wind_direction_10_m_above_gnd,
        "wind_speed_80_m_above_gnd": input_data.wind_speed_80_m_above_gnd,
        "wind_direction_80_m_above_gnd": input_data.wind_direction_80_m_above_gnd,
        "wind_gust_10_m_above_gnd": input_data.wind_gust_10_m_above_gnd,
        "angle_of_incidence": input_data.angle_of_incidence,
        "zenith": input_data.zenith,
        "azimuth": input_data.azimuth
    }])

    # Scale the input data
    input_scaled = scaler.transform(input_df)

    # Make prediction
    prediction = model.predict(input_scaled)
    return prediction[0]

# API endpoints
@app.post("/predict/rf")
async def predict_rf(input_data: InputData):
    try:
        prediction = predict(input_data, rf_model)
        return {"predicted_power": float(prediction)}
    except Exception as e:
        return {"error": str(e)}

@app.post("/predict/gb")
async def predict_gb(input_data: InputData):
    try:
        prediction = predict(input_data, gb_model)
        return {"predicted_power": float(prediction)}
    except Exception as e:
        return {"error": str(e)}

@app.post("/predict/ann")
async def predict_ann(input_data: InputData):
    try:
        prediction = predict(input_data, ann_model)
        return {"predicted_power": float(prediction)}
    except Exception as e:
        return {"error": str(e)}

# Start ngrok to expose the app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Run the FastAPI app with uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\uvicorn\server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\uvicorn\main.py", line 579, in run
    server.run()
  File "C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\uvicorn\server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "C:\Users\Munawwar Shamim\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 133, in _run_once
    

Public URL: NgrokTunnel: "https://fa31-39-51-55-239.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [16260]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "OPTIONS /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "OPTIONS /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "OPTIONS /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "OPTIONS /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "OPTIONS /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 200 OK
INFO:     39.51.55.239:0 - "POST /predict/gb HTTP/1.1" 